# Uniview module for Supernovae over time

Data downloaded from The Open Supernova Catalog https://sne.space on Aug. 20, 2019

*Aaron Geller, 2019*

### Imports and function definitions

In [1]:
#This directory contains all the data needed for the module. It should be in the same directory as the notebook
dataFolder = "data" 
import sys, os, shutil, errno, string, urllib


sys.path.append(( os.path.abspath( os.path.join(os.path.realpath("__file__"),  os.pardir, os.pardir) )))
import uvmodlib.v1 as uvmod

In [2]:
# # Run this if you want to build in 3.0 rather than 2.0
# from config import Settings
# Settings.uvcustommodulelocation='C:\\Users\\msubbarao\\SCISS\\Uniview Theater 3.0\\Custom Modules'

### USES Conf Template 

In [39]:
Template = """mesh
{  
    data SNdata    ./modules/$folderName/OpenSNCatConverted.raw

    cullRadius $cr
    glslVersion 330
    
    propertyCollection 
    {        
        __objectName__
        {           

            vec1f radiusScale 1 | public | desc "point size scale" | widget slider | range 0 1000
            vec1f eventTime 2020 | public | desc "event time " | widget slider | range -1000 11800
            
        }
    }

    ############# to hold the time information for SN
    renderTexture
    {
        name                      stateTexture
        width                     1
        height                    1
        numTextures               1
        isPingPong                true
        isPersistent              true
        isFramePersistent         true
        internalTextureFormat     GL_RGB32F
        magnify                   GL_NEAREST
        minify                    GL_NEAREST
    }
    ############# set Transition State
    pass 
    {
        useDataObject quad
        renderTarget
        {
            name stateTexture
            enableColorClear false
        }
         shader
        {
            type defaultMeshShader
            {
                vertexShader   ./modules/$folderName/pass0.vs
                fragmentShader ./modules/$folderName/state.fs
                textureFBO stateTexture stateTexture
                stateManagerVar __objectName__.transitionLength transitionLength 
                stateManagerVar __objectName__.jump jump 
                stateManagerVar __objectName__.eventTime eventTime 
                parameter2f timeRange 1985 2020
           }
        }      
    }
    
    
    ############# SN v Time
    pass
    {
        useDataObject SNdata
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/SNvtime.gs
                vertexShader ./modules/$folderName/SNvtime.vs
                fragmentShader   ./modules/$folderName/SNvtime.fs
                textureFBO stateTexture stateTexture
                texture cmap ./modules/$folderName/cmap.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                    colorspace linear
                }
                
                stateManagerVar __objectName__.radiusScale  radiusScale
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }
}"""

### SN class

In [42]:
class SNClass():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("SNvtime.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("SNvtime.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("SNvtime.fs",dataFolder)
        self.cr = 1000
        self.Scale = 1

    def generatemod(self):
        self.object.setgeometry(self.object.name+"Mesh.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = self.object.name+"Mesh.usesconf"
        s = string.Template(Template)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir,
                                 cr = self.cr,
                                 Scale = self.Scale
                                 ))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

### Object Instantiation

In [43]:
model = SNClass(uvmod.OrbitalObject())
scene = uvmod.Scene()
parentScene = uvmod.Scene()
modinfo = uvmod.ModuleInformation()
generator = uvmod.Generator()

### Specify Settings and generate the module

In [44]:
scene.setname("SNvTime")
scene.setparent("Extragalactic")
scene.setunit(1)
scene.setentrydist(1e5)

model.object.setcameraradius(1)
model.object.setcoord(scene.name)
model.object.setname("SNvTime")
model.object.setguiname("/KavliLecture/Zooniverse/SNvTime")
model.object.settargetradius(100)
model.object.showatstartup(False)
model.cr = 10

modinfo.setname("SNvTime")
modinfo.setauthor("Aaron Geller<sup>1</sup>, Laura Trouille<sup>1</sup>, Chris Lintott<sup>2</sup> <br />(1)Adler Planetarium,<br />(2)Oxford University,")
modinfo.cleardependencies()
modinfo.setdesc("Uniview module for a SN vs time.  Data are from The Open Supernova Catalog, https://sne.space .")
#modinfo.setthumbnail("data/M82ScreenshotThumb.png")
modinfo.setversion("1.0")

generator.generate("SNvTime",[scene],[model],modinfo)
uvmod.Utility.senduvcommand(model.object.name+".reload")


## To reload

In [30]:
from config import Settings

In [37]:
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, Settings.uvcustommodulelocation+'\\'+model.object.name)
uvmod.Utility.senduvcommand(model.object.name+".reload")

### Create colormap texture

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

In [4]:
gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))

def plot_cmap(colormap):
    fig=plt.imshow(gradient, aspect=1, cmap=colormap)
    plt.axis('off')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    plt.savefig("data/cmap.png", bbox_inches='tight',pad_inches=0)

plot_cmap('tab20')